**Import libraries and import data set.**

In [2]:
import pandas as pd
import numpy as np
import sedc_algorithm
from function_edc import fn_1 
import scipy

In [3]:
%run sedc_algorithm.py #run sedc_algorithm.py module

**For this demonstration, we use the [Movielens 1M data set](https://grouplens.org/datasets/movielens/1m/), which contains movie viewing behavior of users. The target variable is binary (taking value 1 if gender = 'MALE' and 0 if gender = 'FEMALE').**

In [5]:
target = pd.read_csv('target_ML1M.csv')
data = pd.read_csv('data_ML1M.csv')
feature_names = pd.read_csv('feature_names_ML1M.csv')

**Split data into a training and test set (80-20%). We use the finetuned MLP hyperparameter configuration as found in the paper of De Cnudde et al. (2018) titled *'An exploratory study towards applying and demystifying deep learning classification on behavioral big data'*. We train the MLP classifier on the training data set.** 

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scipy.sparse.csr_matrix(data.iloc[:,1:3707].values), target.iloc[:,1], test_size=0.2, random_state=0)

In [7]:
from sklearn.neural_network import MLPClassifier
MLP_model = MLPClassifier(activation='relu', learning_rate_init=0.30452, alpha=0.0001, learning_rate='adaptive', early_stopping=True, hidden_layer_sizes=(532,135,1009), solver='lbfgs', batch_size=100)
MLP_model.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=100, beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(532, 135, 1009), learning_rate='adaptive',
              learning_rate_init=0.30452, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

**Calculate the Area under the ROC curve (AUC) of the model on the test set.**

In [8]:
from sklearn.metrics import roc_auc_score

Scores = MLP_model.predict_proba(x_test)[:,1] #predict scores using the trained MLP model
AUC = roc_auc_score(y_test,Scores) #output AUC of the model 
print("The AUC of the model is %f" %AUC)

The AUC of the model is 0.816161


**Predict 25% of the test instances as positive (gender = 'MALE') (e.g., because of a limited target budget). Obtain the indices of the test instances that are predicted as 'MALE', i.e. the instances that the model is most sure of that they are 'MALE' users.**

In [9]:
probs = MLP_model.predict_proba(x_test)[:,1]
threshold_classifier_probs = np.percentile(probs,75) 
predictions_probs = (probs>=threshold_classifier_probs)
indices_probs_pos = np.nonzero(predictions_probs)#indices of the test instances that are positively-predicted

In [10]:
classification_model = MLP_model 

def classifier_fn(X):
    c=classification_model.predict_proba(X)
    y_predicted_proba=c[:,1]
    return y_predicted_proba

**Create an SEDC explainer object. By default, the SEDC algorithm stops looking for explanations when a first explanation is found or when a 2-minute time limit is exceeded or when more than 50 iterations are required (see edc_agnostic.py for more details). Only the active (~nonzero) features are perturbed (~set to zero) to evaluate the impact on the model's predicted output. In other words, only the movies that a user has watched can become part of the counterfactual explanation of the model prediction.**

In [11]:
explainer_SEDC = SEDC_Explainer(feature_names = np.array(feature_names.iloc[:,1]), 
                               threshold_classifier = threshold_classifier_probs, 
                               classifier_fn = classifier_fn)

**Show indices of positively-predicted test instances.** 

In [12]:
indices_probs_pos #all instances that are predicted as 'MALE'

(array([   4,   21,   25,   30,   32,   57,   60,   61,   62,   65,   75,
          82,   84,   86,   88,   93,   94,   95,   97,  101,  102,  106,
         116,  120,  122,  123,  127,  133,  141,  142,  144,  148,  150,
         152,  153,  162,  164,  166,  167,  173,  174,  175,  180,  181,
         189,  190,  199,  200,  206,  211,  214,  220,  221,  222,  223,
         230,  238,  239,  254,  255,  257,  263,  264,  269,  271,  273,
         282,  284,  287,  290,  292,  295,  296,  298,  301,  302,  305,
         307,  309,  312,  316,  317,  319,  339,  340,  341,  343,  349,
         350,  352,  353,  358,  360,  361,  372,  375,  384,  394,  395,
         396,  401,  415,  417,  421,  436,  451,  454,  460,  466,  471,
         472,  475,  478,  482,  485,  487,  489,  498,  501,  503,  510,
         513,  523,  525,  529,  532,  533,  548,  550,  554,  557,  563,
         564,  565,  572,  575,  579,  587,  590,  591,  600,  609,  615,
         617,  623,  627,  640,  641, 

**Explain why the user with index = 4 is predicted as a 'MALE' user by the model.**

In [13]:
index = 4
instance_idx = x_test[index]
explanation = explainer_SEDC.explanation(instance_idx)

Initialization is complete.

 Elapsed time 0 


 Iteration 1 

Length of new_combinations is 1 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 250 


 Iteration 2 

Length of new_combinations is 2 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 373 


 Iteration 3 

Length of new_combinations is 3 features.
New combination cannot be expanded

 Elapsed time 0 


 Size combis to expand 373 

Iterations are done.

 Elapsed time 0 



In [14]:
explanation = explainer_SEDC.explanation(instance_idx)

Initialization is complete.

 Elapsed time 0 


 Iteration 1 

Length of new_combinations is 1 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 250 


 Iteration 2 

Length of new_combinations is 2 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 373 


 Iteration 3 

Length of new_combinations is 3 features.
New combination cannot be expanded

 Elapsed time 0 


 Size combis to expand 373 

Iterations are done.

 Elapsed time 0 



**Show explanation(s) that is/are found.**

In [15]:
explanation[0]

[['Austin Powers', 'American Pie (1999)', 'Platoon (1986)']]

In [16]:
print("IF the user did not watch the movie(s) " + str(explanation[0][0]) + ", THEN the predicted class would change from 'MALE' to 'FEMALE'.")


IF the user did not watch the movie(s) ['Austin Powers', 'American Pie (1999)', 'Platoon (1986)'], THEN the predicted class would change from 'MALE' to 'FEMALE'.


**Show more information about the explanation(s): *explanation[0]* shows the explanation set(s), *explanation[1]* shows the number of active features of the instance to explain, *explanation[2]* shows the number of explanations found, *explanation[3]* shows the number of features in the smallest-sized explanation, *explanation[4]* shows the time elapsed in seconds to find the explanation, *explanation[6]* shows the predicted score change when removing the feature(s) in the smallest-sized explanation, *explanation[7]* shows the number of iterations that the algorithm needed.**

In [17]:
explanation

([['Austin Powers', 'American Pie (1999)', 'Platoon (1986)']],
 126,
 11,
 3,
 0.8140113353729248,
 [array([7.47539586e-10])],
 3)

**Explain why the user with index = 30 is predicted as a 'MALE' user by the model.**

In [18]:
index = 30
instance_idx = x_test[index]
explanation = explainer_SEDC.explanation(instance_idx)

Initialization is complete.

 Elapsed time 0 


 Iteration 1 

Length of new_combinations is 1 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 382 


 Iteration 2 

Length of new_combinations is 2 features.
New combinations can be expanded

 Elapsed time 1 


 Size combis to expand 571 


 Iteration 3 

Length of new_combinations is 3 features.
New combination cannot be expanded

 Elapsed time 1 


 Size combis to expand 571 

Iterations are done.

 Elapsed time 1 



**Show the smallest-sized explanation found by the SEDC explainer for user with index = 30.**

In [19]:
print("IF the user did not watch the movie(s) " + str(explanation[0][0]) + ", THEN the predicted class would change from 'MALE' to 'FEMALE'.")

IF the user did not watch the movie(s) ['Star Wars', 'Apocalypse Now (1979)', 'Dr. Strangelove or'], THEN the predicted class would change from 'MALE' to 'FEMALE'.


**Show the 10 first explanation(s) found by the SEDC algorithm to explain the user index = 30. We change max_explained to 10.**

In [20]:
explainer_SEDC2 = SEDC_Explainer(feature_names = np.array(feature_names.iloc[:,1]), 
                               threshold_classifier = threshold_classifier_probs, 
                               classifier_fn = classifier_fn, max_explained=10)

In [24]:
index = 30
instance_idx = x_test[index]
explanation = explainer_SEDC2.explanation(instance_idx)

Initialization is complete.

 Elapsed time 0 


 Iteration 1 

Length of new_combinations is 1 features.
New combinations can be expanded

 Elapsed time 0 


 Size combis to expand 382 


 Iteration 2 

Length of new_combinations is 2 features.
New combinations can be expanded

 Elapsed time 1 


 Size combis to expand 571 


 Iteration 3 

Length of new_combinations is 3 features.
New combination cannot be expanded
While loop 0
Length of new_combinations is 189 features.
Score_predicted minus new_score is 0.000000.

 Elapsed time 2 


 Size combis to expand 569 


 Iteration 4 

Length of new_combinations is 3 features.
New combination cannot be expanded

 Elapsed time 3 


 Size combis to expand 569 

Iterations are done.

 Elapsed time 3 



**There are 85 explanations found after 3 iterations. The first 10 are shown. The time elapsed is 1.07 seconds. The number of active features (movies watched) is 192 movies.**

In [25]:
explanation

([['Star Wars', 'Moonraker (1979)', 'Dr. Strangelove or'],
  ['Star Wars', 'Apocalypse Now (1979)', 'Dr. Strangelove or'],
  ['Star Wars', 'Apocalypse Now (1979)', 'Moonraker (1979)'],
  ['Star Wars', 'Apocalypse Now (1979)', 'Being There (1979)'],
  ['Star Wars', 'Moonraker (1979)', 'Being There (1979)'],
  ['Star Wars', 'Moonraker (1979)', 'Matrix'],
  ['Star Wars', 'Apocalypse Now (1979)', 'Matrix'],
  ['Star Wars', 'Moonraker (1979)', 'From Russia with Love (1963)'],
  ['Star Wars', 'Apocalypse Now (1979)', 'From Russia with Love (1963)'],
  ['Star Wars', 'Apocalypse Now (1979)', 'GoodFellas (1990)']],
 192,
 14,
 3,
 3.104339122772217,
 [array([4.05321554e-10]),
  array([4.02575528e-10]),
  array([3.16998872e-10]),
  array([3.15935278e-10]),
  array([3.07406989e-10]),
  array([2.88289614e-10]),
  array([2.82577295e-10]),
  array([2.60298894e-10]),
  array([2.58034261e-10]),
  array([2.53876031e-10])],
 4)